In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
# Load the dataset
from google.colab import files
uploaded = files.upload()
file_path = 'ipc_sections.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to get an overview
df.head()

Saving ipc_sections.csv to ipc_sections (1).csv


,Description,Offense,Punishment,Section
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,IPC_140
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,IPC_127
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,IPC_128
3,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,IPC_129
4,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,IPC_130


In [8]:
import numpy as np
import pandas as pd

# Load the dataset
file_path = 'ipc_sections.csv'
df = pd.read_csv(file_path)

# Extract relevant columns for input features and target variable
X = df[['Offense','Punishment']].values  # Input features
y = df[['Description','Section', 'Punishment']].values  # Target variable

# Print the first few rows of X and y to verify the data
print("Input Features (X):")
print(X[:5])

print("\nTarget Variable (y):")
print(y[:5])

Input Features (X):
[['Wearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman'
  '3 Months or Fine or Both']
 ['Receiving property taken by war or depredation mentioned in sections 125 And 126'
  '7 Years + Fine + forfeiture of property']
 ['Public servant voluntarily allowing prisoner of State or war in his custody to escape'
  'Imprisonment for Life or 10 Years + Fine']
 ['Public servant negligently suffering prisoner of State or war in his custody to escape'
  'Simple Imprisonment 3 Years + Fine']
 ['Aiding escape of, rescuing or harbouring, such prisoner, or offering Any resistance to the recapture of such prisoner'
  'Imprisonment for Life or 10 Years + Fine']]

Target Variable (y):
[['Description of IPC Section 140\nAccording to section 140 of Indian penal code, Whoever, not being a soldier, sailor or airman in the Military, Naval or Air service of the Government of India, wears a

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
# Check the data types of 'Offense' and 'Punishment'
print("Data Types:")
print("Offense:", X[:, 0].dtype)
print("Punishment:", X[:, 1].dtype)

# Convert 'Offense' and 'Punishment' to strings if they are not already
X[:, 0] = X[:, 0].astype(str)
X[:, 1] = X[:, 1].astype(str)

# Combine 'Offense' and 'Punishment' into a single text feature
X_text = X[:, 0] + ' ' + X[:, 1]

# Continue with the rest of the code...


# Combine 'Offense' and 'Punishment' into a single text feature
X_text = X[:, 0] + ' ' + X[:, 1]

# Use CountVectorizer to convert text data to numerical format
vectorizer = CountVectorizer()
X_numerical = vectorizer.fit_transform(X_text)

# Use LabelEncoder to convert the target variable to numerical format
# Combine columns into a single column
y_combined = df['Description'].astype(str) + "" + df['Section'].astype(str) + "" + df['Punishment'].astype(str)
label_encoder = LabelEncoder()
y_numerical = label_encoder.fit_transform(y_combined)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_numerical, y_numerical, test_size=0.2, random_state=42)

# Print the shape of the numerical features and target variable
print("Shape of X_numerical:", X_numerical.shape)
print("Shape of y_numerical:", y_numerical.shape)

Data Types:
Offense: object
Punishment: object
Shape of X_numerical: (444, 992)
Shape of y_numerical: (444,)


In [10]:
from keras.models import Sequential
from keras.layers import Dense

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_numerical)), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │          63,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 442)                 │          14,586 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 80,218 (313.35 KB)

 Trainable params: 80,218 (313.35 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder  # Add this import
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Load the dataset
file_path = 'ipc_sections.csv'
df = pd.read_csv(file_path)

# Extract relevant columns for input features and target variable
X = df[['Offense', 'Punishment']].astype(str).values  # Convert to string
y = df[['Description','Section','Punishment']].values  # Target variable

# Convert labels to numerical format using LabelEncoder
# Combine columns into a single column
y_combined = df['Description'].astype(str) + "" + df['Section'].astype(str) + "" + df['Punishment'].astype(str)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_combined)

# Number of unique classes in your dataset
output_neurons = len(np.unique(y))
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine 'Offense' and 'Punishment' into a single text feature
X_text_train = X_train[:, 0] + ' ' + X_train[:, 1]
X_text_test = X_test[:, 0] + ' ' + X_test[:, 1]

# Use CountVectorizer to convert text data to numerical format
vectorizer = CountVectorizer()
X_train_numerical = vectorizer.fit_transform(X_text_train)
X_test_numerical = vectorizer.transform(X_text_test)

# Compute sample weights for balancing classes
sample_weights = compute_sample_weight('balanced', y_train)

# Define the neural network model with adjustments
model = Sequential()
model.add(Dense(128, input_dim=X_train_numerical.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Set the number of output neurons to match the unique classes
model.add(Dense(output_neurons, activation='softmax'))

# Compile the model with sample weights
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model with sample weights
history_weighted = model.fit(X_train_numerical.toarray(), y_train, epochs=1000, batch_size=100, validation_split=0.2, sample_weight=sample_weights)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_numerical.toarray(), y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch 1/1000


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.0000e+00 - loss: 6.1001 - val_accuracy: 0.0000e+00 - val_loss: 6.0929
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0018 - loss: 6.0863 - val_accuracy: 0.0000e+00 - val_loss: 6.1006
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0140 - loss: 6.0754 - val_accuracy: 0.0000e+00 - val_loss: 6.1099
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0030 - loss: 6.0522 - val_accuracy: 0.0000e+00 - val_loss: 6.1202
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0055 - loss: 6.0359 - val_accuracy: 0.0000e+00 - val_loss: 6.1320
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0085 - loss: 6.0215 - val_accuracy: 0.0000e+00 - val_loss: 6.1451
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.0048 - loss: 5.9967 - val_accuracy: 0.0000e+00 - val_loss: 6.1605
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0158 - loss: 5.9794 - val_

In [26]:
import pandas as pd
import numpy as np
from tabulate import tabulate

# Your specific input
input_text = "wine lootery and robbery and murder"
input_text1 = input_text.split(" and ")  # Split the input into separate words/phrases

# Prepare a list to store the predictions
predictions = []

# Loop through each word/phrase in the input_text1 list
for text in input_text1:
    # Use CountVectorizer to convert each word/phrase to numerical format
    input_numerical = vectorizer.transform([text])

    # Make a prediction using the trained model
    prediction = model.predict(input_numerical.toarray())

    # If you want the predicted class label (assuming one-hot encoding is used)
    predicted_label = np.argmax(prediction)

    # If you want to convert the predicted label back to the original label (e.g., IPC code)
    predicted_label_original = label_encoder.inverse_transform([predicted_label])

    # Append the prediction results for each word/phrase to the predictions list (without class index)
    predictions.append({
        "Input Text": text,
        "Predicted IPC": predicted_label_original[0]  # Only store the predicted IPC label
    })

# Create a DataFrame to show the results in table form
output_df = pd.DataFrame(predictions)

# Use tabulate to print the table in a more stylized format
print(tabulate(output_df, headers='keys', tablefmt='grid', showindex=False))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Input Text   | Predicted IPC                                                                                                                                                                                                                                                                                                                                                 

In [28]:
model.save('text_prediction_model.keras')